In [1]:
from keras.models import Sequential
from keras.layers import Flatten, Dense
import pandas as pd
from pprint import pprint
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
from keras.applications.inception_v3 import InceptionV3
import keras.backend as K

Using TensorFlow backend.


In [2]:
df = pd.read_csv('./data2/driving_log.csv', header=None)
df.head()

,0,1,2,3,4,5,6
0,data2/IMG/center_2020_03_16_00_29_48_632.jpg,data2/IMG/left_2020_03_16_00_29_48_632.jpg,data2/IMG/right_2020_03_16_00_29_48_632.jpg,0.0,0.0,0.0,0.000078
1,data2/IMG/center_2020_03_16_00_29_48_735.jpg,data2/IMG/left_2020_03_16_00_29_48_735.jpg,data2/IMG/right_2020_03_16_00_29_48_735.jpg,0.0,0.0,0.0,0.000079
2,data2/IMG/center_2020_03_16_00_29_48_837.jpg,data2/IMG/left_2020_03_16_00_29_48_837.jpg,data2/IMG/right_2020_03_16_00_29_48_837.jpg,0.0,0.0,0.0,0.000078
3,data2/IMG/center_2020_03_16_00_29_48_949.jpg,data2/IMG/left_2020_03_16_00_29_48_949.jpg,data2/IMG/right_2020_03_16_00_29_48_949.jpg,0.0,0.0,0.0,0.000079
4,data2/IMG/center_2020_03_16_00_29_49_055.jpg,data2/IMG/left_2020_03_16_00_29_49_055.jpg,data2/IMG/right_2020_03_16_00_29_49_055.jpg,0.0,0.0,0.0,0.000078


In [3]:
center_img_paths = df[0].values.tolist()# + df[1].values.tolist() + df[2].values.tolist()
x_train = np.array([mpimg.imread(img_path.strip()) for img_path in center_img_paths])
y_train = np.array(df[3].values.tolist())

In [4]:
x_train_flip = np.array([np.fliplr(img) for img in x_train])
y_train_flip = -y_train.copy()

In [5]:
input_size = 139
inception = InceptionV3(weights='imagenet', include_top=False,
                        input_shape=(input_size,input_size,3))

In [6]:
for layer in inception.layers:
    layer.trainable = False

In [7]:
from keras.layers import Input, Lambda
# import tensorflow as tf

# Makes the input placeholder layer 160x320x3
sim_input = Input(shape=(160, 320, 3))

# Re-sizes the input with Kera's Lambda layer & attach to sim_input
def resize_method(image):
    import tensorflow as tf
    return tf.image.resize_images(image, (138, 139))
    
resized_input = Lambda(resize_method)(sim_input)

# Feeds the re-sized input into Inception model
# You will need to update the model name if you changed it earlier!
inp = inception(resized_input)

In [8]:
fc1 = Dense(256, activation='relu')(Flatten()(inp))
predictions = Dense(1)(fc1)

In [9]:
# Imports the Model API
from keras.models import Model
from keras.optimizers import Adam
# Creates the model, assuming your final layer is named "predictions"
model = Model(inputs=sim_input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(lr=0.0001), loss='mse')

# Check the summary of this new model to confirm the architecture
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 160, 320, 3)       0         
_________________________________________________________________
lambda_1 (Lambda)            (None, 138, 139, 3)       0         
_________________________________________________________________
inception_v3 (Model)         multiple                  21802784  
_________________________________________________________________
flatten_1 (Flatten)          (None, 12288)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               3145984   
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
Total params: 24,949,025
Trainable params: 3,146,241
Non-trainable params: 21,802,784
________________________________________________________

In [10]:
hist = model.fit(x_train, y_train, validation_split=0.2, shuffle=True, batch_size=32, epochs=2)

Train on 4876 samples, validate on 1220 samples
Epoch 1/2
4876/4876 [==============================] - 221s 45ms/step - loss: 0.3392 - val_loss: 0.0914
Epoch 2/2
4876/4876 [==============================] - 217s 45ms/step - loss: 0.1228 - val_loss: 0.0803


In [15]:
model.save('model_inception.h5')